# First Assignment - Working with Dependency Graphs  

- **Full name**: Matteo Farina  
- **Mat. Number**: 221252  
- **email**: [matteo.farina-1@studenti.unitn.it](mailto:matteo.farina-1@studenti.unitn.it)

### Content  
  
This notebook contains the code and the explanations concerning the *mandatory* part of the first assignment. Specifically, students were asked to work with [spaCy](https://spacy.io/) and define functions to:   
- [Task 1](#Task-1): extract a path of dependency relations from ROOT to a token;  
- [Task 2](#Task-2): extract the subtree of dependents given a token;  
- [Task 3](#Task-3): check if a given list of tokens (segment of a sentence) forms a subtree;  
- [Task 4](#Task-4): identify the head of a span, given its tokens;  
- [Task 5](#Task-5): extract sentence subject, direct object and indirect object spans.  
  
Each task will be presented with a brief explanation of both the goal to achieve and the implemented strategies. Then, each function associated to a task will be presented in the respective code cell. Finally, at the very bottom of each code cell, the core function will be tested and its output will be printed to show you what has actually been achieved.

### Requirements  
The code in this notebook runs under the **Python v3.9** interpreter. Python dependencies has been managed with *Conda*, but you may use other tools such as *venv* or *virtualenv* as well.
  
As mentioned in the previous section, **spaCy** is the only library that has to be manually installed to run the code in this notebook:   
- To install it, run `pip install spacy`  
  
Then, also the spacy english model needs to  be installed on your machine. This will be necessary in order to instantiate and perform all the text-processing pipelines across the notebook:  
- To install spacy's english model, run `python -m spacy download en_core_web_sm`  
  
The code cell below instantiates the default example sentences that will be used for testing purposes as well as the text-processing pipeline for the english language.

In [33]:
import spacy

# load the sentences
examples = []
with open("data/mandatory/examples.txt", 'r') as f:
    [examples.append(line.strip()) for line in f.readlines()]

# create the pipeline (will be referenced as a global variable throughout the notebook)
nlp = spacy.load('en_core_web_sm')

# show sents
print("Example sentences: ", examples, sep="\n")

Example sentences: 
['I saw the man with a telescope.', 'My mom gives me a wonderful gift.', 'We take Natural Language Understanding labs every tuesday.']


### Task 1  
**Description**: "extract a path of dependency relations from ROOT to a token"  
  
**Strategy**: In order to achieve the goal of the task, which is to define a function that parses a sentence, gets the dependency graph and returns the list of dependency relations from root, I found it useful to split the task into three sub-tasks as follows:  

1. **build the list of tokens from ROOT to a given token based on the dependency graph**. This is achieved by the *relpath* function. Starting from a token, it climbs the dependency tree in a bottom-up fashion jumping from head to head. The climbing stops when the root node is met, producing the path from a token to root. Then, this path can simply be reversed to obtain the path from root to the given token.  
2. **map each token in the retrieved path to its dependency relation wrt its head**. This is achieved by leveraging spaCy tokens' *dep_* attribute inside the *deprelslist* function and eventually produces the list of dependency relations from root to a given token.
3. **do the same for every token in a sentence**. A sentence is parsed using the spacy's default pipeline for english, so tokens are accessible. Then, the actions described in the first two steps are applied to each token and the output (list of dependency relations for each token) is produced.

In [34]:
def relpath(token, mode='bottom-up'):
    """
    Build the backwards path from a token to the root node and returns the list of tokens.
    If :param token: is the ROOT, the returned list will contain only :param token: itself.
    Params:
    - :param token: the token from which to build the path
    - :param mode: direction to follow when returning the path ('bottom-up' --> from token to root 
    or 'top-down' --> from root to token)
    """     
    # check data types
    from spacy.tokens import Token
    assert type(token) == Token, "'token' argument of 'relpath' must be of type 'spacy.tokens.Token'"
    assert mode in ('bottom-up', 'top-down'), "'mode' argument must be one of ('bottom-up', 'top-down')"
    
    # return list to be filled
    path = [token]
    
    # build the path
    token_ = token
    while token_.dep_ != 'ROOT':
        head = token_.head
        path.append(head)
        token_ = head
        
    # return path from token to ROOT or from ROOT to token according to :param mode:
    return path if mode=='bottom-up' else path[::-1] 
    
def deprelslist(token):
    """
    Extract a path of dependency relations from ROOT to :param token:
    Params:
    - :param token: token up to which to build the dependency list
    Returns:
    - :deplist: list containing relations from the ROOT node to the TOKEN node.
    """
    # check data types
    from spacy.tokens import Token
    assert type(token) == Token, "'token' argument of 'deprels' must be of type 'spacy.tokens.Token'"
    
    # grab the branch from root to token
    token_list = relpath(token, mode='top-down')
    
    # map each token to its dependency relation and return
    return [t.dep_ for t in token_list]

def deprels(sentence):
    """
    Extract the path of dependency relations for each token in :param sentence:.
    :return: list of lists. return(i) = list of dependency relations of the i-th token of :param sentence:
    """
    global nlp
    doc = nlp(sentence)
    return [deprelslist(token) for token in doc]
        
# test the fn for the complete sentence
for sentence in examples:
    print("\n", sentence, deprels(sentence), sep="\n")



I saw the man with a telescope.
[['ROOT', 'nsubj'], ['ROOT'], ['ROOT', 'dobj', 'det'], ['ROOT', 'dobj'], ['ROOT', 'prep'], ['ROOT', 'prep', 'pobj', 'det'], ['ROOT', 'prep', 'pobj'], ['ROOT', 'punct']]


My mom gives me a wonderful gift.
[['ROOT', 'nsubj', 'poss'], ['ROOT', 'nsubj'], ['ROOT'], ['ROOT', 'dative'], ['ROOT', 'dobj', 'det'], ['ROOT', 'dobj', 'amod'], ['ROOT', 'dobj'], ['ROOT', 'punct']]


We take Natural Language Understanding labs every tuesday.
[['ROOT', 'nsubj'], ['ROOT'], ['ROOT', 'dobj', 'compound', 'compound'], ['ROOT', 'dobj', 'compound'], ['ROOT', 'dobj', 'compound'], ['ROOT', 'dobj'], ['ROOT', 'npadvmod', 'det'], ['ROOT', 'npadvmod'], ['ROOT', 'punct']]


### Task 2  
**Descripton**: "extract the subtree of dependents given a token"  
  
**Strategy**: also for what concerns this task, I thought it could've been useful to 'divide and conquer'. Specifically, the task is addressed with two steps, one that acts on individual units (token-level) and the other that extends it to work on a sentence-level.  
1. **retrieve the list of nodes in the subtree of a token**. This is achieved through the *subtree* function, where the spaCy tokens' *subtree* attribute (a generator) is iterated through. The default spaCy's behaviour yields tokens in the exact order they appear in the original sentence. Also, it yields a list with the token itself in case it doesn't have dependents. These features have not been modified, since they were part of the task's specifications.  
2. **do the same for each token in a sentence**. To achieve the final goal, a sentence is parsed using the spacy pipeline for the english language. Thus, tokens are accessible so we can iterate through them and get each one's subtree. The output is then returned (list of lists, where internal lists are made of tokens constituting a subtree).

In [35]:
def subtree(token):
    """
    Extracts the subtree of the dependents of :param token: and returns them inside a list.
    Tokens are ordered as they appear in the original sentence (spaCy default).
    When a token has no dependents, a list containing only the token itself is returned.
    """
    from spacy.tokens import Token
    assert type(token) == Token, "'token' argument of 'subtree' must be of type 'spacy.tokens.Token'"
    return [t for t in token.subtree]

def subtrees(sentence):
    """
    Returns every subtree of a sentence inside a list. 
    Params:  
    - :param sentence: input sentence (str)  
    """
    assert type(sentence) == str, "'sentence' argument of 'subtrees' must be of type 'str'"
    global nlp
    doc = nlp(sentence)
    return [subtree(token) for token in doc]

# test things out
for sentence in examples:
    print("\n", sentence, subtrees(sentence), sep="\n")



I saw the man with a telescope.
[[I], [I, saw, the, man, with, a, telescope, .], [the], [the, man], [with, a, telescope], [a], [a, telescope], [.]]


My mom gives me a wonderful gift.
[[My], [My, mom], [My, mom, gives, me, a, wonderful, gift, .], [me], [a], [wonderful], [a, wonderful, gift], [.]]


We take Natural Language Understanding labs every tuesday.
[[We], [We, take, Natural, Language, Understanding, labs, every, tuesday, .], [Natural], [Natural, Language], [Understanding], [Natural, Language, Understanding, labs], [every], [every, tuesday], [.]]


### Task 3  
**Description**: "check if a given list of tokens (segment of a sentence) forms a subtree"  
  
**Strategy**: to check if a list of tokens forms a subtree, a procedure could be checking it against all the possible subtrees of a sentence, as they are represented as lists as well. Therefore, the logic previously implemented in the *subtrees* function comes handy. However, using the exact same function implies generating all possible subtrees before even checking against the first one. For this reason, wanting to allow for early checks and saving computational time as a best practice, a *generator-based subtrees* function has been purposefully implemented.  
  
To sum up, the **algorithm is so implemented**:  
>1. the input sentence is parsed using the spacy pipeline for the english language  
2. start iterating through tokens  
   2.1. generate token's subtree and compare it to the input list. Stop if a match is found.

In [36]:
def subtrees_gen(sentence):
    """
    Exact same behaviour of 'subtrees(sentence)', but yields values instead of returning them.
    """
    assert type(sentence) == str, "'sentence' argument of 'subtrees' must be of type 'str'"
    global nlp
    doc = nlp(sentence)
    for token in doc:
        yield subtree(token)

def is_subtree(sentence, lot):
    """
    Check if a given list of tokens forms a subtree.
    Params:  
    - :param sentence: sentence of reference
    - :param lot: 'list-of-tokens' to be used to check whether they form a subtree or not
    :return: True if :param lot: is a subtree of :param sentence:, False otherwise
    """
    assert type(sentence) == str, "'sentence' argument of 'is_subtree' must be of type 'str'"
    assert type(lot) == list, "'lot' argument of 'is_subtree' must be of type 'list'"

    # generate subtrees and compare the ordered list of tokens in the subtree with the one to check against
    for stree in subtrees_gen(sentence):
        stree_list = [t.text for t in stree]
        if stree_list == lot:
            return True
    return False

# test things out
lot_true = ['with', 'a', 'telescope']  # is actually a subtree of the 1st example sentence
lot_false = ['mom', 'gives', 'me']  # is not a subtree of the 2nd example sentence
print("Is {} a subtree in sentence '{}'? {}".format(lot_true, examples[0], is_subtree(examples[0], lot_true)))
print("Is {} a subtree in sentence '{}'? {}".format(lot_false, examples[1], is_subtree(examples[1], lot_false)))

Is ['with', 'a', 'telescope'] a subtree in sentence 'I saw the man with a telescope.'? True
Is ['mom', 'gives', 'me'] a subtree in sentence 'My mom gives me a wonderful gift.'? False


### Task 4  
**Description**: "identify the head of a span, given its tokens"  
  
**Strategy**: this task can be addressed by correctly using the *Span.root* attribute of Spacy's span objects. The correct usage is thereafter enforced with a type check. This way, we can safely get the root of any slice of a sentence.  
  
**Disclaimer**: because on Piazza it has been written that the input of this function should be "a sequence of words (not necessarily a sentence)", the basic implementation below doesn't account for situations where we may want to find the root of a *list of tokens* (where 'list' means python list). Also, I couldn't implement a function that takes as input a common python string, because in that case I would need the parent doc to be passed to the function as well in order to locate the string inside the doc and build the span, but passing also the doc wasn't allowed by the task specifications.  
  
So, to sum up:
- `head("with a telescope")` will raise an `AssertionError` due to type constraints;  
- `head(["with", "a", "telescope"])` will also raise and `AssertionError` due to type constraints;  
- `head(doc[start_idx:end_idx])` will successfully return the head of the span from `start_idx` (included) to `end_idx` (excluded).

In [37]:
def head(span):
    """
    Given a span (string representing a contiguous sequence of words in a sentence), return its head.
    """
    from spacy.tokens.span import Span
    assert type(span) == Span, "'span' argument of 'head' must be of type 'spacy.tokens.span.Span'"
    return span.root

# test things out
docs = [nlp(sent) for sent in examples]
span_one = docs[0][4:7]  # 'with a telescope'
span_two = docs[1][4:7]  # 'a wonderful gift'
print("Head of '{}' is '{}'".format(span_one, head(span_one)))
print("Head of '{}' is '{}'".format(span_two, head(span_two)))

Head of 'with a telescope' is 'with'
Head of 'a wonderful gift' is 'gift'


### Task 5  
**Description**: "extract sentence subject, direct object and indirect object spans"  
  
**Strategy**: this task is addressed by leveraging the *dep_* property of spaCy tokens as well as the *subtree* function defined for the second task.  
1. A sentence is at first parsed with the spacy pipeline for the english language.  
2. Then, tokens and their relations are retrieved in such a way that they are pairwise indexed. This means that `relations[i]` contains a string defining the dependency relation that `tokens[i]` has with respect to its head. In this way, I am able to directly index tokens by their dependency relation and the usage of python's built-in `list.index(elem)` method comes handy.  
3. Once that tokens of interests have been retrieved (*nsubj*, *dobj* and *iobj*), their subtree is retrieved too. Obviously, no subtree is retrieved for dependency relations that do not appear in the parsed sentence, so a default empty list is initialized in that case.  
4. The algorithm then returns a dictionary with three keys ('nsubj', 'dobj', 'iobj') where the respective values are the spans represented by the retrieved subtrees. So, overall, the output is a dict-of-lists. 

**Note**: in case of non-projective parses, the subtrees aren't guaranteed to provide a *contiguous* list of tokens. The correct identification of the entities of interest is not affected, though.

In [38]:
def extract(sentence):
    """
    Extract sentence subject, direct object and indirect object spans (if it exists, empty list otherwise.)
    :param sentence: input sentence to get the data from
    :return: dict containing exactly three keys ('nsubj', 'dobj', 'iobj') with the respective spans
    Example:
    extract("I saw a man.") --> {'nsubj': ['I'], 'dobj': ['a', 'man'], 'iobj': []}
    """
    assert type(sentence) == str, "'sentence' argument of 'extract' must be of type 'str'."
    global nlp
    doc = nlp(sentence)
    
    # get equally indexed tokens and relations
    tokens = [token for token in doc]
    relations = [token.dep_ for token in tokens]
    
    # build the span of the nominal subject
    nsubj = []
    if 'nsubj' in relations:
        nsubj = subtree(tokens[relations.index('nsubj')])
    
    # build the span of the direct object
    dobj = []
    if 'dobj' in relations:
        dobj = subtree(tokens[relations.index('dobj')])
        
    # build the span of the indirect object (spaCy == 'dative')
    iobj = []
    if 'dative' in relations:
        iobj = subtree(tokens[relations.index('dative')])
        
    return {'nsubj': nsubj, 'dobj': dobj, 'iobj': iobj}

print(extract(examples[0]))  # no iobj
print(extract(examples[1]))  # with iobj
print(extract(examples[2]))  # more complex sentence
print(extract("Jet Blue canceled our flight this morning which was already late."))  # non projective sent.

{'nsubj': [I], 'dobj': [the, man], 'iobj': []}
{'nsubj': [My, mom], 'dobj': [a, wonderful, gift], 'iobj': [me]}
{'nsubj': [We], 'dobj': [Natural, Language, Understanding, labs], 'iobj': []}
{'nsubj': [Jet, Blue], 'dobj': [our, flight, which, was, already, late], 'iobj': []}
